<div class="frontmatter text-center">
<h1> Introduction to Data Science and Programming</h1>
<h2>Class 23: Graph properties</h2>
<h3>IT University of Copenhagen, Fall 2019</h3>
<h3>Instructor: Michael Szell</h3>
</div>

# Source
This notebook was adapted from:
* Bruno Gonçalves / Data4Sci: https://github.com/DataForScience/Networks

In [ ]:
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt

# Graph class

We now integrate our prefered graph representation (adjacency dict) into a class that we can build on. For now we provide it with just placeholders for our data

In [ ]:
class Graph:
    def __init__(self, directed=False):
        self._nodes = {}
        self._edges = {}
        self._directed = directed

For ease of explanation, we will be adding methods to this class as we progress. To allow for this in a convenient way, we must declare a Python decorator that will be in charge of modifying the class as we implement further functionality.

Understanding this function is not important for the scope of the lecture, but if you are curious, you can find more information on [Decorators](https://www.python.org/dev/peps/pep-0318/) and [setattr](https://docs.python.org/3/library/functions.html#setattr) in the offical Python documentation.

In [ ]:
def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

We can already instantiate our skeleton class:

In [ ]:
G = Graph()

and verify that it has nothing hiding inside other than the default Python methods and the fields we defined

In [ ]:
?dir

In [ ]:
dir(G)

## Nodes

Now we add our first utility methods. *add_node* will be responsible for adding a single node to the Graph, while *add_nodes_from* will prove useful to add nodes in bulk. We can also add node attributes by passing keyword arguments to any of these two functions. Here is a detailed explanation of keyword arguments: https://realpython.com/python-kwargs-and-args/ In short, `**kwargs` allows to unpack a dictionary.

In [ ]:
@add_method(Graph)
def add_node(self, node, **kwargs):
    self._nodes[node] = kwargs

In [ ]:
@add_method(Graph)
def add_nodes_from(self, nodes, **kwargs):
    for node in nodes:
        if isinstance(node, tuple):
            self._nodes[node[0]] = node[1:]
        else:
            self._nodes[node] = kwargs

Let's check that it works as promised:

In [ ]:
G.add_node("A", color="blue")

In [ ]:
G._nodes

In [ ]:
G.add_node("Z", color="green", size=14)

In [ ]:
G._nodes

In [ ]:
G.add_nodes_from("ABC", color='red')

In [ ]:
G._nodes

Here it is important to note 2 things:

- Since add_nodes_from expects the first argument to be a list of nodes, it treated each character of the string as an individual node
- By adding the same node twice we overwrite the previous version.

# Edges

Now we add the equivalent functionality for edges.

In [ ]:
@add_method(Graph)
def add_edge(self, node_i, node_j, **kwargs):
    if node_i not in self._nodes:
        self.add_node(node_i)
    
    if node_j not in self._nodes:
        self.add_node(node_j)
    
    if node_i not in self._edges:
        self._edges[node_i] = {}
        
    if node_j not in self._edges[node_i]:
        self._edges[node_i][node_j] = {}
        
    self._edges[node_i][node_j] = kwargs
    
    if not self._directed:
        if node_j not in self._edges:
            self._edges[node_j] = {}

        if node_i not in self._edges[node_j]:
            self._edges[node_j][node_i] = {}

        self._edges[node_j][node_i] = kwargs
        
@add_method(Graph)
def add_edges_from(self, edges, **kwargs):
    for edge in edges:
        self.add_edge(*edge, **kwargs)

Before we proceed, let us create a new Graph object:

In [ ]:
G = Graph()

In [ ]:
G._directed

And add the edges from the edge list we considered before

In [ ]:
edge_list = [
    ('A', 'B'),
    ('A', 'C'),
    ('A', 'E'),
    ('B', 'C'),
    ('C', 'D'),
    ('C', 'E'),
    ('D', 'E')]

In [ ]:
G.add_edges_from(edge_list)

And we can easily check that it looks correct, both for nodes and edges

In [ ]:
G._nodes

In [ ]:
G._edges

For Completeness, we add a function to return a list of all the edges and their attributes (if any)

In [ ]:
@add_method(Graph)
def edgelist(self):
    e = []
    
    for node_i in self._edges:
        for node_j in self._edges[node_i]:
            e.append([node_i, node_j, self._edges[node_i][node_j]])
            
    return e

So we recover the undirected version of the edge list we started with

In [ ]:
G.edgelist()

## Graph properties

Now that we have a minimally functional Graph object, we can start implementing functionality to retrieve information about the Graph.

### Node information

Obtaining the number of nodes:

In [ ]:
@add_method(Graph)
def number_of_nodes(self):
    return len(self._nodes)

So we confirm that we have 5 nodes as expected

In [ ]:
G.number_of_nodes()

And to retrieve the degree of each node one must check the number of corresponding entries in the edge dictionary

In [ ]:
@add_method(Graph)
def degrees(self):
    deg = {}
    
    for node in self._nodes:
        if node in self._edges:
            deg[node] =  len(self._edges[node])
        else:
            deg[node] = 0
    
    return deg

With the expected results

In [ ]:
G.degrees()

### Edge Information

The number of edges is given by:

In [ ]:
@add_method(Graph)
def number_of_edges(self):
    n_edges = 0
    
    for node_i in self._edges:
        n_edges += len(self._edges[node_i])
    
    # If the graph is undirected, don't double count the edges
    if not self._directed:
        n_edges /= 2
    
    return n_edges

And so we find, as expected

In [ ]:
G.number_of_edges()

We also add a method to check if the graph id directed

In [ ]:
@add_method(Graph)
def is_directed(self):
    return self._directed

In [ ]:
G.is_directed()

### Weights

As we saw, each edge can potentially have a weight associated with it (it defaults to 1). We also provide a function to recover a dictionary mapping edges to weights

In [ ]:
@add_method(Graph)
def weights(self, weight="weight"):
    w = {}
    
    for node_i in self._edges:
        for node_j in self._edges[node_i]:
            if weight in self._edges[node_i][node_j]:
                w[(node_i, node_j)] = self._edges[node_i][node_j][weight]
            else:
                w[(node_i, node_j)] = 1
    return w

As we didn't explicitly include any weight information in our graph, we find that all the weights are 1

In [ ]:
G._edges['A']['B']['weight']=4

In [ ]:
G.weights()

### Topology and Correlations

One particularly useful property of a graph is the list of nearest neighbors of a given node. With our formulation, this is particularly convenient to implement:

In [ ]:
@add_method(Graph)
def neighbours(self, node):
    return list(self._edges[node].keys())

So we find that node C has nearest neighbours nodes A, B, D, E:

In [ ]:
G.neighbours('C')

We are also intersted in the degree and weight distributions. Before we can compute them, we define a utility function to generate a probability distribution from a dictionary of values:

In [ ]:
@add_method(Graph)
def _build_distribution(data, normalize=True):
    values = data.values()
    dist = list(Counter(values).items())
    dist.sort(key=lambda x:x[0])
    dist = np.array(dist, dtype='float')
           
    if normalize:
        norm = dist.T[1].sum()
        dist.T[1] /= norm
    
    return dist

By default the probability distribution is normalized such that the sum of all values is 1. Using this utility function we can now calculate the degree distribution:

In [ ]:
@add_method(Graph)
def degree_distribution(self, normalize=True):
    deg = self.degrees()
    dist = Graph._build_distribution(deg, normalize)
    
    return dist

The degree distribution for our Graph is then:

In [ ]:
G.degree_distribution(False)

Where we can see that we have 2 nodes of both degree 2 and 3 and 1 of degree 4.

Similarly, for the weight distribution:

In [ ]:
@add_method(Graph)
def weight_distribution(self, normalize=True):
    deg = self.weights()
    dist = Graph._build_distribution(deg, normalize)
    
    return dist


And we confirm that five edges have weight 1, while one edge has weight 4.

In [ ]:
G.weight_distribution()

We now calculate the average degree of the nearest neighbours for each node.

In [ ]:
@add_method(Graph)
def neighbour_degree(self):
    knn = {}
    deg = self.degrees()
    
    for node_i in self._edges:
        NN = self.neighbours(node_i)
        total = [deg[node_j] for node_j in NN]
        knn[node_i] = np.mean(total)
        
    return knn

In [ ]:
G.neighbour_degree()

And the distribution by degree:

In [ ]:
@add_method(Graph)
def neighbour_degree_function(self):
    knn = {}
    count = {}
    deg = self.degrees()
    
    for node_i in self._edges:
        NN = self.neighbours(node_i)
        total = [deg[node_j] for node_j in NN]
        
        curr_k = deg[node_i]
        knn[curr_k] = knn.get(curr_k, 0) + np.mean(total)
        count[curr_k] = count.get(curr_k, 0) + 1
        
    for curr_k in knn:
        knn[curr_k]/=count[curr_k]
    
    knn = list(knn.items())
    knn.sort(key=lambda x:x[0])
    
    return np.array(knn)

From which we obtain:

In [ ]:
G.neighbour_degree_function()

# Zachary Karate Club

Let's now look at an empirical Graph, from J. Anthro. Res. 33, 452 (1977)

For convenience, we load the data from a file using numpy

In [ ]:
edges = np.loadtxt('karate.txt')

Now we can use the functions defined above to generate the corresponding graph

In [ ]:
Karate = Graph()

In [ ]:
Karate.add_edges_from(edges)

Our graph has 34 nodes

In [ ]:
Karate.number_of_nodes()

And 78 edges

In [ ]:
Karate.number_of_edges()

The degree distribution is:

In [ ]:
Pk = Karate.degree_distribution()

In [ ]:
Pk

Which we can plot:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.bar(Pk.T[0], Pk.T[1])
axes.set_ylim([0,0.35])
axes.set_xlim([0,18])
axes.set_xlabel('k')
axes.set_ylabel('P(k)')
axes.set_title("Degree distribution of Zachary Karate Club");

The average degree of the nearest neighbours as a function of the degree is:

In [ ]:
knn = Karate.neighbour_degree_function()

Which we plot as well

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.plot(knn.T[0], knn.T[1], '-o')
axes.set_xlim([0,18])
axes.set_ylim([0,17])
axes.set_xlabel('$k$')
axes.set_ylabel('$k_{nn}(k)$')
axes.set_title("Average neighbor degrees of Zachary Karate Club");

Conclusion: knn(k) is decreasing. This is not normal in social networks and could show that something was "wrong", leading to the split.

***
To wrap up this lecture, let's save the current state of our Graph class. For this we use some Jupyter Notebook magic. It's not important to understand this, but you can read about it in the [Jupyter notebook](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html) documentation.

In [ ]:
def export_class(path, filename):
    import io
    from nbformat import read

    with io.open(path, 'r', encoding='utf-8') as f:
        nb = read(f, 4)

    fp = open(filename, "wt")

    for cell in nb.cells:
        if cell.cell_type == 'code':
            first_line = cell.source.split('\n')[0]
            
            if "class " in first_line or "add_method" in first_line:
                print(cell.source, file=fp)

                print("\n", file=fp)
            elif "import" in first_line:
                for line in cell.source.split('\n'):
                    if not line.startswith("%"):
                        print(line.strip(), file=fp)
                        
                print("\n", file=fp)

    fp.close()

After this line, we'll have a Python module called "Graph.py" containing all the methods in our Graph class.

In [ ]:
export_class('class23_graphproperties.ipynb', 'Graph.py')